In [3]:
import re

In [4]:
class Valve():
    
    def __init__(self, line):
        m = re.match('Valve (..) has flow rate=(\d+); tunnels? leads? to valves? (.+)$', line)
        
        self.name = m.group(1)
        self.rate = int(m.group(2))
        
        self.tunnels = [t.strip() for t in m.group(3).split(',')]
        
        self.distances = {}
        
        self.maxflow = 0
        self.max_lvl = 0
        

In [5]:
def map_dists(start, valves, rel_valves):
    
    nvisited = set(valves.keys())
    valve_dist = {valve:1e9 for valve in valves.keys()}
    
    valve_dist[start] = 0
    curv = start
    
    while len(nvisited) != 0:
        
        for t in valves[curv].tunnels:
            if valve_dist[t] > valve_dist[curv] + 1:
                valve_dist[t] = valve_dist[curv] + 1
        nvisited.remove(curv)
        
        mind = 1e9
        for v in nvisited:
            if valve_dist[v] < mind:
                curv = v
                mind = valve_dist[v]
                
    for v in rel_valves:
        valves[start].distances[v] = valve_dist[v]

In [6]:
def map_flows(n_closed, flow, rtime, start, rel_valves, valves):
    
    nc = set(n_closed)
    nc.remove(start)
    t_used = 0
    if valves[start].rate != 0:
        t_used = 1
        
    if rtime < 0:
        return

    flow_now = valves[start].rate * (rtime - t_used) + flow

    if valves[start].maxflow < flow_now:
        valves[start].maxflow = flow_now
        # if valves[start].max_lvl >= lvl:
        #     return
        # else:
        #    valves[start].max_lvl = lvl
    
    for nv in nc:
        map_flows(nc, flow_now, (rtime - t_used - valves[start].distances[nv]), nv, rel_valves, valves)

In [13]:
def map_flows_with_elephant(n_closed, flow, rtime1, rtime2, p1, p2, rel_valves, valves, 
                            openvalves={}, openset=[], lvl=0):
    
    t_used = 0
    
    # print(lvl, 'I', p1, 'elephant', p2, flow, rtime1, rtime2, n_closed)
    
    if rtime1 < 0 and rtime2 < 0:
        return
    
    if rtime1 >= rtime2:
        # Move me
        i = 1
        for nv in n_closed:
            nc = set(n_closed)
            os = [v for v in openset]
            os.append(nv)
            os = tuple(sorted(os))
            nc.remove(nv)
            if lvl == 0:
                print('I from', p1, nv, len(n_closed)-i, 'left to check')
            i+=1
            
            rtime1_now = rtime1 - valves[p1].distances[nv] - 1
            if rtime1_now < 0:
                continue
            flow_now = valves[nv].rate * (rtime1_now) + flow

            if os in openvalves:
                if openvalves[os] > flow_now:
                    continue
                else:
                    openvalves[os] = flow_now
                    
            if valves[nv].maxflow < flow_now:
                valves[nv].maxflow = flow_now
            map_flows_with_elephant(nc, flow_now, rtime1_now, rtime2, nv, p2, rel_valves, valves, openvalves, os, lvl+1)        
    else:
        # Move elephant
        i=1
        for nv in n_closed:
            nc = set(n_closed)
            os = [v for v in openset]
            os.append(nv)
            os = tuple(sorted(os))
            nc.remove(nv)
            if lvl == 1:
                print('Elephant', p2, nv, len(n_closed)-i, 'left to check')
            i+=1
            
            rtime2_now = rtime2 - valves[p2].distances[nv] - 1
            if rtime2_now < 0:
                continue
        
            flow_now = valves[nv].rate * (rtime2_now) + flow

            if os in openvalves:
                if openvalves[os] > flow_now:
                    continue
                else:
                    openvalves[os] = flow_now
                    
            if valves[nv].maxflow < flow_now:
                valves[nv].maxflow = flow_now
            map_flows_with_elephant(nc, flow_now, rtime1, rtime2_now, p1, nv, rel_valves, valves, openvalves, os, lvl+1)


In [17]:
valves = {}
with open('input16.txt', 'r') as inf:
    for line in inf:
        valve = Valve(line.strip())
        valves[valve.name] = valve

rel_valves = ['AA']

for n,v in valves.items():
    if v.rate != 0:
        rel_valves.append(n)

# Now find the distances between the relevant valves:

for rv in rel_valves:
    # Find distance to all other relevant valves
    map_dists(rv, valves, rel_valves)
    
nclosed = set(rel_valves)

map_flows(nclosed, 0, 30, 'AA', rel_valves, valves)   
  
maxflow = 0

for v in rel_valves:
    print(valves[v].name, valves[v].maxflow)
    if valves[v].maxflow > maxflow:
        maxflow = valves[v].maxflow

print('Maxflow', maxflow)

nclosed = set(rel_valves)
nclosed.remove('AA')

for v in rel_valves:
    valves[v].maxflow = 0

maxflow = 0
map_flows_with_elephant(nclosed, 0, 26, 26, 'AA', 'AA', rel_valves, valves)   

print('Maxflow1', maxflow)
for v in rel_valves:
    print(valves[v].name, valves[v].maxflow)
    if valves[v].maxflow > maxflow:
        maxflow = valves[v].maxflow

print('Maxflow', maxflow)

  


AA 0
LC 1568
BX 1673
HX 1626
JI 1588
HF 1515
OK 1530
UN 1575
GB 1638
XM 1616
OH 1673
GV 1565
CQ 1523
GR 1615
TS 1620
IR 1619
Maxflow 1673
I from AA BX 14 left to check
Elephant AA JI 13 left to check
Elephant AA HF 12 left to check
Elephant AA OH 11 left to check
Elephant AA CQ 10 left to check
Elephant AA LC 9 left to check
Elephant AA TS 8 left to check
Elephant AA GR 7 left to check
Elephant AA UN 6 left to check
Elephant AA GV 5 left to check
Elephant AA HX 4 left to check
Elephant AA GB 3 left to check
Elephant AA XM 2 left to check
Elephant AA OK 1 left to check
Elephant AA IR 0 left to check
I from AA JI 13 left to check
Elephant AA BX 13 left to check
Elephant AA HF 12 left to check
Elephant AA OH 11 left to check
Elephant AA CQ 10 left to check
Elephant AA LC 9 left to check
Elephant AA TS 8 left to check
Elephant AA GR 7 left to check
Elephant AA UN 6 left to check
Elephant AA GV 5 left to check
Elephant AA HX 4 left to check
Elephant AA GB 3 left to check
Elephant AA XM 2 le